In [1]:
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz' -O julia.tar.gz
    tar -x -f julia.tar.gz -C /usr/local --strip-components 1
    rm julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; add BenchmarkTools; add CuArrays; precompile;"'
echo 'Done'


  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
Precompiling project...
Done


In [2]:
!julia -e '\
import Pkg                                                         ;\
Pkg.update()                                                       ;\
Pkg.add("DataFrames")                                              ;\
Pkg.add("CSV")                                                     ;\'

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


# Julia DataFrames.jl 介紹

# Day 017 作業：載入 COVID-19 資料集

今天的作業將使用 `DataFrames.jl` 及 `CSV.jl` 套件，來載入美國約翰霍普金斯大學提供的 COVID-19 資料集 (2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE)。資料集作為教育及研究使用，並且被用來建立視覺化儀表板提供檢視及追蹤 COVID-19 疫情狀況。

資料集 GitHub: [https://github.com/CSSEGISandData/COVID-19](https://github.com/CSSEGISandData/COVID-19)

資料集格式為 CSV，整合不同資料來源，主要分為 Daily Report 及時間序列資料。

請自行下載 2020/4/2 的 daily report資料集：[https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/04-02-2020.csv](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/04-02-2020.csv)

Daily Report 主要欄位有：

- Province/State: 省名 (中國)、州名或市名 (美國、加拿大、澳洲)、或是事件名稱 (例如鑽石公主號)...
- Country/Region: 國名或區域名
- Last Update: 最後更新日期/時間，格式為 24 小時制的 UTC 時間
- Confirmed: 確診案例
- Deaths: 死亡案例
- Recovered: 康復案例
- Lat, Long: 經緯度
- Combined Key: 複合 Key 值

請問此資料集的筆數 (row) 及欄位數 (column) 各是多少？

In [3]:
# 彙總 DataFrame 資訊

!julia -e '\
using DataFrames, CSV                                                                                                                         ;\
url =  "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-02-2020.csv"       ;\
df = CSV.read(download(url))                                                                                                                  ;\
println(describe(df))                                                                                                                         ;\'  

12×8 DataFrame
│ Row │ variable       │ mean     │ min           │ median   │ max                     │ nunique │ nmissing │ eltype                  │
│     │ Symbol         │ Union…   │ Any           │ Union…   │ Any                     │ Union…  │ Union…   │ Type                    │
├─────┼────────────────┼──────────┼───────────────┼──────────┼─────────────────────────┼─────────┼──────────┼─────────────────────────┤
│ 1   │ FIPS           │ 30958.4  │ 66            │ 29093.0  │ 99999                   │         │ 281      │ Union{Missing, Int64}   │
│ 2   │ Admin2         │          │ Abbeville     │          │ unassigned              │ 1431    │ 264      │ Union{Missing, String}  │
│ 3   │ Province_State │          │ Alabama       │          │ Zhejiang                │ 135     │ 177      │ Union{Missing, String}  │
│ 4   │ Country_Region │          │ Afghanistan   │          │ Zimbabwe                │ 181     │          │ String                  │
│ 5   │ Last_Update    │         

### 作業1：數值 Column 的計算


(1) 請問截至4月2日為止 (UTC 時間)，全球累計的確診、死亡、及㡽復案例數各是多少？ 【提示】可參考 `sum()` 內建函式。


In [4]:
!julia -e '\
using DataFrames, CSV                                                                                                                         ;\
url =  "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-02-2020.csv"       ;\
df = CSV.read(download(url))                                                                                                                  ;\
println("截至4月2日為止 (UTC 時間) ")                                                                                                           ;\
println("全球累計的確診數     ",sum(df[!,:Confirmed]) )                                                                                        ;\
println("全球累計的死亡數       ",sum(df[!,:Deaths]) )                                                                                         ;\
println("全球累計的㡽復案例數  ",sum(df[!,:Recovered]) )                                                                                        ;\' 

截至4月2日為止 (UTC 時間) 
全球累計的確診數     1013303
全球累計的死亡數       52983
全球累計的㡽復案例數  210263


(2) 列出確診案例數前 10 名的 Province_State, Country_Region, Confirmed, Deaths, Recovered 等 5 個 column。



In [5]:
!julia -e '\
using DataFrames, CSV                                                                                                                         ;\
url =  "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-02-2020.csv"       ;\
df = CSV.read(download(url))                                                                                                                  ;\
println( first( sort(df, [:Confirmed], rev = true)[ : , [:Province_State, :Country_Region, :Confirmed, :Deaths, :Recovered]], 10 ) )          ;\' 

10×5 DataFrame
│ Row │ Province_State │ Country_Region │ Confirmed │ Deaths │ Recovered │
│     │ String⍰        │ String         │ Int64     │ Int64  │ Int64     │
├─────┼────────────────┼────────────────┼───────────┼────────┼───────────┤
│ 1   │ missing        │ Italy          │ 115242    │ 13915  │ 18278     │
│ 2   │ missing        │ Spain          │ 112065    │ 10348  │ 26743     │
│ 3   │ missing        │ Germany        │ 84794     │ 1107   │ 22440     │
│ 4   │ Hubei          │ China          │ 67802     │ 3199   │ 63471     │
│ 5   │ missing        │ France         │ 59105     │ 5387   │ 12428     │
│ 6   │ New York       │ US             │ 51809     │ 1397   │ 0         │
│ 7   │ missing        │ Iran           │ 50468     │ 3160   │ 16711     │
│ 8   │ missing        │ United Kingdom │ 33718     │ 2921   │ 135       │
│ 9   │ missing        │ Switzerland    │ 18827     │ 536    │ 4013      │
│ 10  │ missing        │ Turkey         │ 18135     │ 356    │ 415       │


### 作業2：找出特定的 Row

截至4月2日為止 (UTC 時間)，台灣的確診、死亡、及㡽復案例數為多少？

【提示】使用點運算 `.==` 來比較同一 column 中所有值。

In [7]:
!julia -e '\
using DataFrames, CSV                                                                                                                         ;\
url =  "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-02-2020.csv"       ;\
df = CSV.read(download(url))                                                                                                                  ;\
println( df[ df[!, :Country_Region] .== "Taiwan*", [ :Country_Region, :Last_Update, :Confirmed, :Deaths, :Recovered] ] )                      ;\'  

1×5 DataFrame
│ Row │ Country_Region │ Last_Update  │ Confirmed │ Deaths │ Recovered │
│     │ String         │ String       │ Int64     │ Int64  │ Int64     │
├─────┼────────────────┼──────────────┼───────────┼────────┼───────────┤
│ 1   │ Taiwan*        │ 4/2/20 11:09 │ 339       │ 5      │ 45        │
